In [10]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import json
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.neighbors import KNeighborsClassifier
from scipy import sparse
import gensim
from gensim.matutils  import Sparse2Corpus
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer


# conda install -c anaconda gensim 
# conda install -c conda-forge lightgbm 
#import lightgbm as lgb

In [11]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/wtf/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
#wordnet_lemmatizer = WordNetLemmatizer()
#wordnet_lemmatizer.lemmatize('dogs')

The following code will help to throw away all HTML tags from article content/title.

In [13]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [14]:
PATH_TO_DATA = './data'

Supplementary function to read a JSON line without crashing on escape characters.

In [15]:
def read_json_line(line=None):
    result = None
    try:        
        result = json.loads(line)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(str(e).split(' ')[-1].replace(')',''))      
        # Remove the offending character:
        new_line = list(line)
        new_line[idx_to_replace] = ' '
        new_line = ''.join(new_line)     
        return read_json_line(line=new_line)
    return result

This function takes a JSON and forms a txt file leaving only article titles. When you resort to feature engineering and extract various features from articles, a good idea is to modify this function.

#### Fields available in the json_data:

```js
['_id', '_timestamp', '_spider', 'url', 'domain', 'published',
 'title', 'content', 'author', 'image_url', 'tags', 'link_tags', 'meta_tags']
```

### Extract tags

In [16]:
def extract_tags_from_json(path_to_inp_json_file, path_to_out_txt_file, total_length):
    '''
    :param path_to_inp_json_file: path to a JSON file with train/test data
    :param path_to_out_txt_file: path to extracted features (here titles), one per line
    :param total_length: we'll pass the hardcoded file length to make tqdm even more convenient
    '''
    with open(path_to_inp_json_file, encoding='utf-8') as inp_file, \
         open(path_to_out_txt_file, 'w', encoding='utf-8') as out_file:
        for line in tqdm_notebook(inp_file, total=total_length):
            json_data = read_json_line(line)
            content = json_data['content'].replace('\n', ' ').replace('\r', ' ')
            tags_str = []
            soup = BeautifulSoup(content, 'lxml')
            try:
                tag_block = soup.find('ul', class_='tags')
                tags = tag_block.find_all('a')
                for tag in tags:
                    tags_str.append(tag.text.translate({ord(' '):None, ord('-'):None}))
                tags = ' '.join(tags_str)
            except Exception:
                tags = 'None'
                
            out_file.write(' '.join(tags_str) + '\n')

In [17]:
%%time
extract_tags_from_json(path_to_inp_json_file=os.path.join(PATH_TO_DATA, 'train.json'),
           path_to_out_txt_file='train_tags.txt', total_length=62313)


CPU times: user 5min 32s, sys: 3.13 s, total: 5min 35s
Wall time: 5min 35s


In [18]:
%%time
extract_tags_from_json(path_to_inp_json_file=os.path.join(PATH_TO_DATA, 'test.json'),
           path_to_out_txt_file='test_tags.txt', total_length=34645)


CPU times: user 3min 5s, sys: 1.45 s, total: 3min 6s
Wall time: 3min 6s


### Extract domain

In [19]:
def extract_domain_from_json(path_to_inp_json_file, path_to_out_txt_file, total_length):
    '''
    :param path_to_inp_json_file: path to a JSON file with train/test data
    :param path_to_out_txt_file: path to extracted features (here titles), one per line
    :param total_length: we'll pass the hardcoded file length to make tqdm even more convenient
    '''
    with open(path_to_inp_json_file, encoding='utf-8') as inp_file, \
         open(path_to_out_txt_file, 'w', encoding='utf-8') as out_file:
        for line in tqdm_notebook(inp_file, total=total_length):
            json_data = read_json_line(line)
            domain = json_data['domain'].replace('\n', ' ').replace('\r', ' ')
            out_file.write(domain + '\n')

In [20]:
%%time
extract_domain_from_json(path_to_inp_json_file=os.path.join(PATH_TO_DATA, 'train.json'),
           path_to_out_txt_file='train_domain.txt', total_length=62313)


CPU times: user 12.5 s, sys: 574 ms, total: 13.1 s
Wall time: 13 s


In [21]:
%%time
extract_domain_from_json(path_to_inp_json_file=os.path.join(PATH_TO_DATA, 'test.json'),
           path_to_out_txt_file='test_domain.txt', total_length=34645)


CPU times: user 6.99 s, sys: 324 ms, total: 7.31 s
Wall time: 7.29 s


### Extract image_url

In [22]:
def extract_image_url_from_json(path_to_inp_json_file, path_to_out_txt_file, total_length):
    '''
    :param path_to_inp_json_file: path to a JSON file with train/test data
    :param path_to_out_txt_file: path to extracted features (here titles), one per line
    :param total_length: we'll pass the hardcoded file length to make tqdm even more convenient
    '''
    with open(path_to_inp_json_file, encoding='utf-8') as inp_file, \
         open(path_to_out_txt_file, 'w', encoding='utf-8') as out_file:
        for line in tqdm_notebook(inp_file, total=total_length):
            json_data = read_json_line(line)
            #tags = json_data['tags'].replace('\n', ' ').replace('\r', ' ')
            #print(json_data['image_url'])
            if json_data['image_url'] != None:
                image_url = '1'
                #image_url = json_data['image_url'].replace('\n', ' ').replace('\r', ' ')
            else:
                image_url = '0'
            out_file.write(image_url + '\n')

In [23]:
%%time
extract_image_url_from_json(path_to_inp_json_file=os.path.join(PATH_TO_DATA, 'train.json'),
           path_to_out_txt_file='train_image_url.txt', total_length=62313)


CPU times: user 12.6 s, sys: 577 ms, total: 13.2 s
Wall time: 13.2 s


In [24]:
%%time
extract_image_url_from_json(path_to_inp_json_file=os.path.join(PATH_TO_DATA, 'test.json'),
           path_to_out_txt_file='test_image_url.txt', total_length=34645)


CPU times: user 6.97 s, sys: 323 ms, total: 7.29 s
Wall time: 7.27 s


### Extract content

In [25]:
def remove_punctuation(text):
    
    # Tokenize the string into words
    #tokens = word_tokenize(text)
    tokens = text.split()

    # Remove non-alphabetic tokens, such as punctuation
    words = [wordnet_lemmatizer.lemmatize(word.lower()) for word in tokens if word.isalpha()]
    #wordnet_lemmatizer = WordNetLemmatizer()
    #wordnet_lemmatizer.lemmatize('dogs')
    return ' '.join(words)

In [26]:
def extract_content_from_json(path_to_inp_json_file, path_to_out_txt_file, total_length):
    '''
    :param path_to_inp_json_file: path to a JSON file with train/test data
    :param path_to_out_txt_file: path to extracted features (here titles), one per line
    :param total_length: we'll pass the hardcoded file length to make tqdm even more convenient
    '''
    with open(path_to_inp_json_file, encoding='utf-8') as inp_file, \
         open(path_to_out_txt_file, 'w', encoding='utf-8') as out_file:
        for line in tqdm_notebook(inp_file, total=total_length):
            json_data = read_json_line(line)
            content = json_data['content'].replace('\n', ' ').replace('\r', ' ')
            content_no_html_tags = remove_punctuation(strip_tags(content))
            out_file.write(content_no_html_tags + '\n')

In [27]:
%%time
extract_content_from_json(path_to_inp_json_file=os.path.join(PATH_TO_DATA, 'train.json'),
           path_to_out_txt_file='train_content.txt', total_length=62313)


CPU times: user 7min 38s, sys: 2.91 s, total: 7min 41s
Wall time: 7min 40s


In [28]:
%%time
extract_content_from_json(path_to_inp_json_file=os.path.join(PATH_TO_DATA, 'test.json'),
           path_to_out_txt_file='test_content.txt', total_length=34645)


CPU times: user 4min 7s, sys: 1.56 s, total: 4min 8s
Wall time: 4min 8s


### Extract titles

In [29]:
def extract_titles_from_json(path_to_inp_json_file, path_to_out_txt_file, total_length):
    '''
    :param path_to_inp_json_file: path to a JSON file with train/test data
    :param path_to_out_txt_file: path to extracted features (here titles), one per line
    :param total_length: we'll pass the hardcoded file length to make tqdm even more convenient
    '''
    with open(path_to_inp_json_file, encoding='utf-8') as inp_file, \
         open(path_to_out_txt_file, 'w', encoding='utf-8') as out_file:
        for line in tqdm_notebook(inp_file, total=total_length):
            json_data = read_json_line(line)
            content = json_data['title'].replace('\n', ' ').replace('\r', ' ')
            #content_no_html_tags = remove_punctuation(strip_tags(content))
            #content_no_html_tags = remove_punctuation(strip_tags(content))
            #out_file.write(content_no_html_tags + '\n')
            out_file.write(content + '\n')

In [30]:
%%time
extract_titles_from_json(path_to_inp_json_file=os.path.join(PATH_TO_DATA, 'train.json'),
           path_to_out_txt_file='train_titles.txt', total_length=62313)


CPU times: user 12.6 s, sys: 575 ms, total: 13.2 s
Wall time: 13.1 s


In [ ]:
%%time
extract_titles_from_json(path_to_inp_json_file=os.path.join(PATH_TO_DATA, 'test.json'),
           path_to_out_txt_file='test_titles.txt', total_length=34645)


CPU times: user 6.99 s, sys: 326 ms, total: 7.32 s
Wall time: 7.32 s


In [ ]:
#!head -5 ./train_titles.txt

### Extract authors

In [ ]:
def extract_authors_from_json(path_to_inp_json_file, path_to_out_txt_file, total_length):
    '''
    :param path_to_inp_json_file: path to a JSON file with train/test data
    :param path_to_out_txt_file: path to extracted features (here titles), one per line
    :param total_length: we'll pass the hardcoded file length to make tqdm even more convenient
    '''
    with open(path_to_inp_json_file, encoding='utf-8') as inp_file, \
         open(path_to_out_txt_file, 'w', encoding='utf-8') as out_file:
        for line in tqdm_notebook(inp_file, total=total_length):
            json_data = read_json_line(line)
            # {'name': None, 'url': 'https://medium.com/@Medium', 'twitter': '@Medium'}
            authors = json_data['author']['url'].replace('\n', ' ').replace('\r', ' ')
            out_file.write(authors + '\n')

In [ ]:
%%time
extract_authors_from_json(path_to_inp_json_file=os.path.join(PATH_TO_DATA, 'train.json'),
           path_to_out_txt_file='train_authors.txt', total_length=62313)


CPU times: user 12.5 s, sys: 572 ms, total: 13 s
Wall time: 13 s


In [ ]:
%%time
extract_authors_from_json(path_to_inp_json_file=os.path.join(PATH_TO_DATA, 'test.json'),
           path_to_out_txt_file='test_authors.txt', total_length=34645)


CPU times: user 7.01 s, sys: 323 ms, total: 7.33 s
Wall time: 7.31 s


### Extract Published

In [ ]:
def extract_published_from_json(path_to_inp_json_file, path_to_out_txt_file, total_length):
    '''
    :param path_to_inp_json_file: path to a JSON file with train/test data
    :param path_to_out_txt_file: path to extracted features (here titles), one per line
    :param total_length: we'll pass the hardcoded file length to make tqdm even more convenient
    '''
    with open(path_to_inp_json_file, encoding='utf-8') as inp_file, \
         open(path_to_out_txt_file, 'w', encoding='utf-8') as out_file:
        for line in tqdm_notebook(inp_file, total=total_length):
            json_data = read_json_line(line)
            # {'$date': '2015-08-03T07:44:50.331Z'}
            published = json_data['published']['$date'].replace('\n', ' ').replace('\r', ' ')
            out_file.write(published + '\n')

In [ ]:
%%time
extract_published_from_json(path_to_inp_json_file=os.path.join(PATH_TO_DATA, 'train.json'),
           path_to_out_txt_file='train_published.txt', total_length=62313)


CPU times: user 12.5 s, sys: 572 ms, total: 13.1 s
Wall time: 13 s


In [ ]:
%%time
extract_published_from_json(path_to_inp_json_file=os.path.join(PATH_TO_DATA, 'test.json'),
           path_to_out_txt_file='test_published.txt', total_length=34645)


CPU times: user 6.96 s, sys: 322 ms, total: 7.28 s
Wall time: 7.27 s


## Add features

### Add Tags features

In [ ]:
cv = TfidfVectorizer(max_features=1000, ngram_range=(1, 1))
#cv = CountVectorizer(ngram_range=(1, 1), max_features=1000)

In [ ]:
%%time
with open('./train_tags.txt') as input_train_file:
    X_train_tags_sparse = cv.fit_transform(input_train_file)

CPU times: user 316 ms, sys: 7.06 ms, total: 324 ms
Wall time: 323 ms


In [ ]:
%%time
with open('./test_tags.txt') as input_train_file:
    X_test_tags_sparse = cv.transform(input_train_file)

CPU times: user 172 ms, sys: 2.65 ms, total: 175 ms
Wall time: 174 ms


### Add Content features

In [ ]:
#stop_words="english"
#tfidf = TfidfVectorizer(max_features=100000, ngram_range=(1, 2), analyzer="word", stop_words="english")
tfidf = TfidfVectorizer(max_features=200000, ngram_range=(1, 3))
# old with 1,2 3 min 7s

In [ ]:
%%time
with open('./train_content.txt') as input_train_file:
    X_train_content_sparse = tfidf.fit_transform(input_train_file)

In [ ]:
%%time
#with open('./test_content.txt', encoding='utf-8') as input_train_file:
with open('./test_content.txt') as input_train_file:
    X_test_content_sparse = tfidf.transform(input_train_file)

### Add title features
We'll use a very simple feature extractor – TfidfVectorizer, meaning that we resort to the Bag-of-Words approach. For now, we are leaving only 50k features.

In [ ]:
#tfidf = TfidfVectorizer(max_features=100000, ngram_range=(1, 2))
tfidf = TfidfVectorizer(max_features=200000, ngram_range=(1, 2))

In [ ]:
%%time
with open('train_titles.txt', encoding='utf-8') as input_train_file:
    X_train_title_sparse = tfidf.fit_transform(input_train_file)

In [ ]:
%%time
with open('test_titles.txt', encoding='utf-8') as input_test_file:
    X_test_title_sparse = tfidf.transform(input_test_file)

In [ ]:
X_train_title_sparse.shape, X_test_title_sparse.shape

Read targets from a file

In [ ]:
train_target = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                        'train_log1p_recommends.csv'), 
                           index_col='id')

In [ ]:
y_train = train_target['log_recommends'].values

Target is still somewhat skewed, though it's allready log1p-transformed (#claps with log1p transformation). Yet, we'll apply log1p once more time.

In [ ]:
plt.hist(y_train, bins=30, alpha=.5, color='red', 
         label='original', range=(0,10));
plt.hist(np.log1p(y_train), bins=30, alpha=.5, color='green', 
         label='log1p', range=(0,10));
plt.legend();

In [ ]:
cv = CountVectorizer(ngram_range=(1, 3), max_features=50000)

In [ ]:
%%time
with open('train_titles.txt', encoding='utf-8') as input_train_file:
    X_train_title_count_sparse = cv.fit_transform(input_train_file)

In [ ]:
%%time
with open('test_titles.txt', encoding='utf-8') as input_test_file:
    X_test_title_count_sparse = cv.transform(input_test_file)

### Add author feature

In [ ]:
train_author_df = pd.read_csv('./train_authors.txt', header=None, names=['author'])
train_author_df.head(3)

In [ ]:
test_author_df = pd.read_csv('./test_authors.txt', header=None, names=['author'])
test_author_df.head(3)

Create a united dataframe of author data for one hot encoding the author data

In [ ]:
# United dataframe of the author data 
full_author_df = pd.concat([train_author_df, test_author_df])

# Index to split the training and test data sets
idx_split = train_author_df.shape[0]

In [ ]:
onehotencoder = OneHotEncoder(categories='auto')
X_full_author_sparse = onehotencoder.fit_transform(full_author_df).toarray()

In [ ]:
X_full_author_sparse.shape

Split the authors back into training and test sets 

In [ ]:
X_train_author_sparse = X_full_author_sparse[:idx_split,:]

In [ ]:
X_test_author_sparse = X_full_author_sparse[idx_split:,:]

In [ ]:
print(X_train_author_sparse.shape)
print(X_test_author_sparse.shape)

In [ ]:
author_onehotencoder = OneHotEncoder(categories='auto', handle_unknown="ignore")

In [ ]:
X_train_author_sparse = author_onehotencoder.fit_transform(train_author_df).toarray()

In [ ]:
X_train_author_sparse.shape

In [ ]:
X_test_author_sparse = author_onehotencoder.transform(test_author_df).toarray()

In [ ]:
X_test_author_sparse.shape

### Add date time features

In [ ]:
train_published_df = pd.read_csv('./train_published.txt',
                           header=None, names=['published'], parse_dates=['published'])
test_published_df = pd.read_csv('./test_published.txt',
                           header=None, names=['published'], parse_dates=['published'])

In [ ]:
def add_time_features(df):    
    hour = df['published'].apply(lambda ts: ts.hour)
    
    morning = ((hour >= 7) & (hour <= 11)).astype('int')
    day = ((hour >= 12) & (hour <= 18)).astype('int')
    evening = ((hour >= 19) & (hour <= 23)).astype('int')
    night = ((hour >= 0) & (hour <= 6)).astype('int')
    
    dayofweek = df['published'].apply(lambda ts: ts.dayofweek)
    is_weekday = ((dayofweek == 5) | (dayofweek == 6)).astype('int')

    onehotencoder = OneHotEncoder(categories='auto')
    dayofweek_hot_encoded = onehotencoder.fit_transform(dayofweek.values.reshape(-1, 1)).toarray()
    
    #onehotencoder = OneHotEncoder(categories='auto')
    #hour_published_hot_encoded = onehotencoder.fit_transform(hour.values.reshape(-1, 1)).toarray()

    #month_published = df['published'].apply(lambda ts: ts.month).astype('int')
    #onehotencoder = OneHotEncoder(categories='auto')
    #month_published_hot_encoded = onehotencoder.fit_transform(month_published.values.reshape(-1, 1)).toarray()

    
    empty_sparse_matrix = csr_matrix((df.shape[0], 0))
    X = hstack([empty_sparse_matrix,
                morning.values.reshape(-1, 1),
                day.values.reshape(-1, 1),
                evening.values.reshape(-1, 1),
                night.values.reshape(-1, 1),
                #dayofweek_hot_encoded,
                is_weekday.values.reshape(-1,1),
                #month_published_hot_encoded # #ignore for now - the test set does not have data for all 12 months
                #hour_published_hot_encoded  
                ]).tocsr()
    return X

In [ ]:
%%time
X_train_time_features_sparse = add_time_features(train_published_df)

In [ ]:
%%time
X_test_time_features_sparse = add_time_features(test_published_df)

### Add image_url feature

In [ ]:
train_image_url_df = pd.read_csv('./train_image_url.txt', header=None, names=['image_url'])
train_image_url_df.head(3)

In [ ]:
test_image_url_df = pd.read_csv('./test_image_url.txt', header=None, names=['image_url'])
test_image_url_df.head(3)

In [ ]:
X_train_image_url_sparse = csr_matrix(train_image_url_df.values)

In [ ]:
X_test_image_url_sparse = csr_matrix(test_image_url_df.values)

### Add domain feature

In [ ]:
train_domain_df = pd.read_csv('./train_domain.txt', header=None, names=['train_domain'])
#train_domain_df.head(15)

In [ ]:
test_domain_df = pd.read_csv('./test_domain.txt', header=None, names=['train_domain'])

In [ ]:
def is_site(x, l):
    if x in l:
      return 1
    return 0


def add_domain_feature(df):    
    foo = pd.DataFrame(index=df.index)
    foo['is_medium'] = df['train_domain'].apply(lambda x: is_site('medium.com', x))
    foo['is_hackernoon'] = df['train_domain'].apply(lambda x: is_site('hackernoon.com', x))
    return csr_matrix(foo.values)

In [ ]:
X_train_domain_sparse = add_domain_feature(train_domain_df)

In [ ]:
X_train_domain_sparse.shape

In [ ]:
X_test_domain_sparse = add_domain_feature(test_domain_df)

In [ ]:
X_test_domain_sparse.shape

### Join all sparse matrices

In [ ]:
%%time
X_train = hstack([X_train_author_sparse,
                  X_train_time_features_sparse,
                  X_train_title_sparse,
                  X_train_content_sparse,
                  X_train_image_url_sparse,
                  X_train_domain_sparse,
                  X_train_tags_sparse,
                  #X_train_title_lda,
                  #X_train_title_count_sparse
                ]).tocsr()

In [ ]:
%%time
X_test = hstack([X_test_author_sparse,
                 X_test_time_features_sparse,
                 X_test_title_sparse,
                 X_test_content_sparse,
                 X_test_image_url_sparse,
                 X_test_domain_sparse,
                 X_test_tags_sparse,
                 #X_test_title_lda
                 #X_test_title_count_sparse
                ]).tocsr()

## Validation and model training
Let's make a 30%-holdout set.

In [ ]:
train_part_size = int(0.7 * train_target.shape[0])
X_train_part = X_train[:train_part_size, :]
y_train_part = y_train[:train_part_size]
X_valid =  X_train[train_part_size:, :]
y_valid = y_train[train_part_size:]

Now we are ready to fit a linear model.

In [ ]:
ridge = Ridge(random_state=17, alpha=1.25, fit_intercept=False, solver="saga")

In [ ]:
%%time
ridge.fit(X_train_part, np.log1p(y_train_part));

After log1p-transformation, we need to apply an inverse  expm1-transformation to predictions.

In [ ]:
ridge_pred = np.expm1(ridge.predict(X_valid))

In [ ]:
ridge_valid_mae = mean_absolute_error(y_valid, ridge_pred)
ridge_valid_mae # 1.052790496860926

In [149]:
#full_sparse_title_count_data =  sparse.vstack([X_train_title_count_sparse, X_test_title_count_sparse])

In [150]:
#Transform our sparse_data to corpus for gensim
#corpus_data_gensim = gensim.matutils.Sparse2Corpus(full_sparse_title_count_data, documents_columns=False)

In [250]:
#%%time
#lda = LdaModel(corpus_data_gensim, num_topics = 30)

In [251]:
#%%time
#def document_to_lda_features(lda_model, document):
#    topic_importances = lda.get_document_topics(document, minimum_probability=0)
#    topic_importances = np.array(topic_importances)
#    return topic_importances[:,1]

#lda_features = list(map(lambda doc:document_to_lda_features(lda, doc),corpus_data_gensim))

In [252]:
#data_pd_lda_features = pd.DataFrame(lda_features)
#data_pd_lda_features.head()

In [186]:
#X_train_title_lda = csr_matrix(data_pd_lda_features.iloc[:y_train.shape[0]])

In [187]:
#X_test_title_lda = csr_matrix(data_pd_lda_features.iloc[y_train.shape[0]:])

In [253]:
#X_train_title_lda.shape

In [254]:
#X_test_title_lda.shape

In [ ]:
#X_train_content_sparse

In [ ]:
#full_sparse_content_count_data =  sparse.vstack([X_train_title_count_sparse, X_test_title_count_sparse])

In [74]:
#knn = KNeighborsClassifier(n_neighbors=50)

In [78]:
#cv = CountVectorizer(ngram_range=(1, 3), max_features=50000)

In [95]:
#%%time
#with open('train_titles.txt', encoding='utf-8') as input_train_file:
#    X_train_title_count_sparse = cv.fit_transform(input_train_file)

In [ ]:
#%%time
#with open('./train_content.txt') as input_train_file:
#    X_train_content_sparse = tfidf.fit_transform(input_train_file)

In [86]:
#baz = X_train_title_count_sparse.toarray()

In [94]:
#y_train

In [92]:
#from sklearn.neighbors import NearestNeighbors

In [93]:
#%%time
#knn.kneighbors(X_train_title_count_sparse)
#knn.fit(baz, y_train)
#knn.fit(X_train_title_count_sparse, y_train)

Then, we fit a LightGBM model with mean_absolute_error as objective (it's important!).

In [88]:
#lgb_x_train_part = lgb.Dataset(X_train_part.astype(np.float32), label=np.log1p(y_train_part))

In [89]:
#lgb_x_valid = lgb.Dataset(X_valid.astype(np.float32), label=np.log1p(y_valid))

In [90]:
#param = {'num_leaves': 255, 
         'objective': 'mean_absolute_error',
         'metric': 'mae'}

In [ ]:
#num_round = 200
#bst_lgb = lgb.train(param, lgb_x_train_part, num_round, valid_sets=[lgb_x_valid], early_stopping_rounds=20)

In [ ]:
lgb_pred = np.expm1(bst_lgb.predict(X_valid.astype(np.float32), num_iteration=bst_lgb.best_iteration))

Let's plot predictions and targets for the holdout set. Recall that these are #recommendations (= #claps) of Medium articles with the  np.log1p transformation.

In [ ]:
plt.hist(y_valid, bins=30, alpha=.5, color='red', label='true', range=(0,10));
plt.hist(ridge_pred, bins=30, alpha=.5, color='green', label='Ridge', range=(0,10));
plt.hist(lgb_pred, bins=30, alpha=.5, color='blue', label='Lgbm', range=(0,10));
plt.legend();

As we can see, the prediction is far from perfect, and we get MAE  ≈  1.3 that corresponds to  ≈  2.7 error in #recommendations.

In [ ]:
ridge_valid_mae = mean_absolute_error(y_valid, ridge_pred)
ridge_valid_mae # 1.0542281512959446

In [ ]:
lgb_valid_mae = mean_absolute_error(y_valid, lgb_pred)
lgb_valid_mae

## Simple blending
Now let's mix predictions. We's just pick up weights 0.6 for Lgbm and 0.4 for Ridge, but these are typically tuned via cross-validation.

In [ ]:
#mean_absolute_error(y_valid, .4 * lgb_pred + .6 * ridge_pred)

Finally, train both models on the full accessible training set, make predictions for the test set and form submission files.

In [ ]:
%%time
ridge.fit(X_train, np.log1p(y_train));

In [ ]:
%%time
ridge_test_pred = np.expm1(ridge.predict(X_test))

In [ ]:
#lgb_x_train = lgb.Dataset(X_train.astype(np.float32), label=np.log1p(y_train))

In [ ]:
#num_round = 60
#bst_lgb = lgb.train(param, lgb_x_train, num_round)

In [ ]:
#lgb_test_pred = np.expm1(bst_lgb.predict(X_test.astype(np.float32)))

Here's a simple mix

In [ ]:
#mix_pred = .4 * lgb_test_pred + .6 * ridge_test_pred

### Leaderboard probing
Now we apply a dirty hack. Originally, we made you come up with it by your own (as a part of an assignment, with only a hint from out side), but now it's described in this tutorial, written within a previous session of mlcourse.ai.

Submitting all zeros gives 4.33328. If you take a pen and a piece of paper and figure out what it means for MAE that all predictions are zeros, then you'll see that it's exactly the mean target value for the test set. We can compare it with mean target for training data and correspondingly adjust predictions. Looks like a dirty hack, however, the same thing is often done with time series prediction (even in production) - merely adjusting your predictions to a change in target variable distribution.

In [ ]:
mean_test_target = 4.33328

In [ ]:
#mix_test_pred_modif = mix_pred + mean_test_target - y_train.mean()

In [ ]:
def write_submission_file(prediction, filename,
                          path_to_sample=os.path.join(PATH_TO_DATA, 'sample_submission.csv')):
    submission = pd.read_csv(path_to_sample, index_col='id')
    
    submission['log_recommends'] = prediction
    submission.to_csv(filename)

In [87]:
# 1.43272
write_submission_file(ridge_test_pred + mean_test_target - y_train.mean(), 'submissions/15-ridge_submission.csv')

In [ ]:
#write_submission_file(mix_test_pred_modif, 'submissions/15-medium-submission-blending.csv')

In [ ]:
#write_submission_file(ridge_test_pred, 'submissions/03-medium-submission.csv') # 2.03034 # no probing

In [ ]:
#write_submission_file(lgb_test_pred + mean_test_target - y_train.mean(), 'submissions/03-lgb_submission.csv')

In [ ]:
#write_submission_file(mix_test_pred_modif, 'submissions/03-medium-submission-probing.csv')

As we can see, simple blending decreases MAE for both holdout predictions and on the leaderboard. However, I don't recommend to play with blending/stacking schemes in the beginning of the competition – it's crucially important to come up with good features first.

You can further improve your model in various ways. I've described them in this kernel. Go and compete, good luck! https://www.kaggle.com/kashnitsky/ridge-countvectorizer-baseline

### TODO implement this part from https://www.kaggle.com/kashnitsky/ridge-and-lightgbm-blending-hacking